# Day 19: Not Enough Minerals

A classic resource and machine building strategy game.

First up, let's have a quick check of the size of the problem:

24 Minutes, with a minute-resolution simulation.
Puzzle input contains 30 blueprints.

The blueprints vary the quantity of the source materials required, but they don't change the type of resource.

## Approach

A first glance, this looks similar to the valves puzzle from a few days ago. I solved that using a brute-force technique, checking every single possible option (and discarding some obvious losers along the way to improve performance).
But there are so many more possibilities here, that taking that approach seems wrong. And also a bit repetative... I want to try something different.

So... how about having a strategy engine that works backwards from the goal?

Generic recipes:
a Ore              -> ore bot   -> Ore
b Ore              -> clay bot  -> Clay
c Ore + d Clay     -> obs bot   -> Obsidian
e Ore + f Obsidian -> geode bot -> Geode

Re-arrange to work backwards: (and ignoring time)
1 Geode = e Ore + f Obsidian
        = e Ore + f (c Ore + d Clay)
        = e Ore + fc Ore + fdb Ore 
        = ( e + fc + fdb ) Ore

Start with 1 ore collecting Robot.
Ore is the primary contraint... but we don't want an ore stock-pile at the end.


Other approaches to consider... could this be turned into a single function that can be optimised? 
What would the inputs be? The choice how many robots of each type to build in each minute.
But it's integer choices... so we're into integer programming - maybe try using: https://www.cvxpy.org/examples/applications/OOCO.html

But perhaps I should try a slightly more straightforward way first?

Many hours in, straightforward isn't working. 

Is this a clue in the puzzle?
"The elephants are starting to look hungry, so you shouldn't take too long; you need to figure out which blueprint would maximize the number of opened geodes after 24 minutes by figuring out which robots to build and when to build them."

Hungry suggests a greedy algorithm.
But already discovered that doens't work.





In [31]:
testData ="""Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""

import re

def process(input:str):
    pattern = '(\d+)'
    for l in input.splitlines():
        match = re.findall(pattern,l)
        blueprint = [int(i) for i in match]
        print(blueprint)

#test
process(testData)

[1, 4, 2, 3, 14, 2, 7]
[2, 2, 3, 3, 8, 3, 12]


In [42]:
ID = 'id'
ORE = 'ore'
CLAY = 'clay'
OBSIDIAN = 'obsidian'
GEODE = 'geoode'
RUNTIME = 24

class Puzzle:
    def blueprintsFrom(self, input):
        pattern = '(\d+)'
        blueprints = []
        for l in input.splitlines():
            match = re.findall(pattern,l)
            intarr = [int(i) for i in match]
            blueprint = {}
            blueprint[ID] = intarr[0]
            blueprint['a'] = intarr[1]
            blueprint['b'] = intarr[2]
            blueprint['c'] = intarr[3]
            blueprint['d'] = intarr[4]
            blueprint['e'] = intarr[5]
            blueprint['f'] = intarr[6]
            blueprints.append(blueprint)
        return blueprints
    
    def __init__(self, input:str):
        self.blueprints = self.blueprintsFrom(input)


class Game:
    def __init__(self, blueprint):
        self.time = 0
        self.materials= {ORE:0, CLAY:0, OBSIDIAN:0, GEODE:0}
        self.bots= {ORE:1, CLAY:0, OBSIDIAN:0, GEODE:0}
        self.blueprint = blueprint

    def playGame(self)->int:
        #play the game for a single blueprint, return the quality score
        print(self.blueprint)
        while self.time < RUNTIME:
            self.tick()

        return self.materials[GEODE] * self.blueprint[ID]

    def tick(self):
        #progres the game 1 minute
        self.time += 1
        startMaterials = self.materials.copy()
        valueOfBots = self.botValues()
        print('Bot value: '+str(valueOfBots))
        startBots = self.bots.copy()

        #build the bots
        sortedBotValue = {k: v for k, v in sorted(valueOfBots.items(), key=lambda item: item[1],reverse=True)}
        
        for bot, val in sortedBotValue.items():
            break
        print((bot,val))
        if val > 0:
            self.buildBot(bot,startMaterials)

        #collect material
        self.collectMaterials(startBots)
        print('T=' + str(self.time)+ ' Bots: ' + str(self.bots) + ' Materials: ' + str(self.materials))




    def botValues(self)->dict:
        #can we calculate the relative bot value?
        #it's the value of an additional bot
        remainingTime = RUNTIME - self.time
        valueOfBot = {}
        if self.canBuildBot(GEODE):
            valueOfBot[GEODE]=1000

        if self.canBuildBot(OBSIDIAN):
            if self.bots[OBSIDIAN]==0:
                valueOfBot[OBSIDIAN]=100
            else:
                #model out the two scenarios, with building the bot and not building the bot... what time do we get to another geode bot?

                #with bot
                timeToGeoBotObsContraint = (self.blueprint['f'] - self.materials[OBSIDIAN])/(self.bots[OBSIDIAN] + 1)
                timeToGeoBotOreConstraint = (self.blueprint['e'] - (self.materials[ORE] - self.blueprint['d']))/self.bots[ORE]
                timeToGoalWithBot = max(timeToGeoBotObsContraint, timeToGeoBotOreConstraint)

                #without bot
                timeToGeoBotObsContraint = (self.blueprint['f'] - self.materials[OBSIDIAN])/(self.bots[OBSIDIAN])
                timeToGeoBotOreConstraint = (self.blueprint['e'] - (self.materials[ORE]))/self.bots[ORE]
                timeToGoalWithoutBot = max(timeToGeoBotObsContraint, timeToGeoBotOreConstraint)

    

                #now need to project impact of these two on remaining time
                estGeoWith = min(remainingTime,timeToGoalWithBot) * self.bots[GEODE] + min(remainingTime,(remainingTime - timeToGoalWithBot -1)) * (self.bots[GEODE]+1)
                estGeoWithout = min(remainingTime,timeToGoalWithoutBot) * self.bots[GEODE] + min(remainingTime,(remainingTime - timeToGoalWithoutBot -1)) * (self.bots[GEODE])

                if estGeoWith > estGeoWithout:
                    valueOfBot[OBSIDIAN] = estGeoWith - estGeoWithout #value in estimated additional geodes
                else: 
                    valueOfBot[OBSIDIAN] = 0
        else:
            valueOfBot[OBSIDIAN] = 0
        
        if self.canBuildBot(CLAY):
            if self.bots[CLAY]==0:
                valueOfBot[CLAY]=100
            else:
                #model out the two scenarios, with building the bot and not building the bot... what time do we get to another geode bot?
                #for both scnarios, we need to consider when an obsidian bot would be built.... can we approx that?
                #yes, because we just want to know if we can get to GEO bot quicker... and so we'd only mine clay if we're going to build 
                #an obsidian bot as quickly as possible.
                #So if we don't want an obsidian bot, we shouldn't bother with a clay bot.
                #So algo:
                # two scanrios: which gets to an obsidian bot quicker
                # plus compare to no obsidian bot and no clay bot
                
                #with bot
                ttObsBotClayContraint = (self.blueprint['d'] - self.materials[CLAY])/(self.bots[CLAY]+1)
                ttObsBotOreContraint = (self.blueprint['c'] - (self.materials[ORE]-self.blueprint['b']))/(self.bots[ORE])
                timeToGoalWithBot = max(ttObsBotClayContraint,ttObsBotOreContraint)

                #without bot
                ttObsBotClayContraint = (self.blueprint['d'] - self.materials[CLAY])/(self.bots[CLAY])
                ttObsBotOreContraint = (self.blueprint['c'] - (self.materials[ORE]))/(self.bots[ORE])
                timeToGoalWithoutBot = max(ttObsBotClayContraint,ttObsBotOreContraint)

                #TODO consider second order effects.
                if timeToGoalWithBot < timeToGoalWithoutBot:
                    valueOfBot[CLAY] = (timeToGoalWithoutBot - timeToGoalWithBot)/self.blueprint['f'] #value in estimated additional geodes
                else:
                    valueOfBot[CLAY] = 0
        else:
            valueOfBot[CLAY] = 0

        if self.canBuildBot(ORE):
            #this one is quite hard, but we can take an approx, based on weighted payback time
            #one ore now is worth 'a' ore in the future 
            #plus all the other things we could do with ore now 
            #let's consider how much ore we actually need
            #1 geode = e + fc+ fdb <- this is the min ore to build robots to make 1 geode
            #to do better than this, we'd need to a) ore to be the constraint; b) there to be enough time to make more ore
            minOre = self.blueprint['e'] + self.blueprint['c'] + self.blueprint['b']
            #apply a discount factor to future extra ore
            extraOreWithBot = (RUNTIME - self.time) - self.blueprint['a']
        
            if self.bots[OBSIDIAN] > 0:
                #model out the two scenarios, with building the bot and not building the bot... what time do we get to another geode bot?

                #with bot
                timeToGeoBotObsContraint = (self.blueprint['f'] - self.materials[OBSIDIAN])/(self.bots[OBSIDIAN] )
                timeToGeoBotOreConstraint = (self.blueprint['e'] - (self.materials[ORE] - self.blueprint['a']))/(self.bots[ORE] +1)
                timeToGoalWithBot = max(timeToGeoBotObsContraint, timeToGeoBotOreConstraint)

                #without bot
                timeToGeoBotObsContraint = (self.blueprint['f'] - self.materials[OBSIDIAN])/(self.bots[OBSIDIAN])
                timeToGeoBotOreConstraint = (self.blueprint['e'] - (self.materials[ORE]))/self.bots[ORE]
                timeToGoalWithoutBot = max(timeToGeoBotObsContraint, timeToGeoBotOreConstraint)

                if timeToGoalWithBot < timeToGoalWithoutBot:
                    valueOfBot[ORE] = timeToGoalWithoutBot - timeToGoalWithBot #value in estimated additional geodes
                else:
                    valueOfBot[ORE] = 0

            else:
                #we don't yet have an obsidian bot... so let's not build another ore bot yet
                valueOfBot[ORE] = 0
        else:
            valueOfBot[ORE] = 0

          

        return valueOfBot
        



    def canBuildBot(self, bot)->bool:
        #try and build a specific bot. return success
        if bot == ORE:
            if self.materials[ORE] >= self.blueprint['a']:
                return True
        elif bot == CLAY:
            if self.materials[ORE] >= self.blueprint['b']: 
                return True
        elif bot == OBSIDIAN:
            if self.materials[ORE] >= self.blueprint['c'] and self.materials[CLAY] >= self.blueprint['d']:
                return True
        elif bot == GEODE:
            if self.materials[ORE] >= self.blueprint['e'] and self.materials[OBSIDIAN] >= self.blueprint['f']:
                return True
        return False

    def buildBot(self, bot, startMaterials)->bool:
        #try and build a specific bot. return success
        if bot == ORE:
            if startMaterials[ORE] >= self.blueprint['a']:
                self.bots[ORE] += 1
                self.materials[ORE] -= self.blueprint['a']
                return True
        elif bot == CLAY:
            if startMaterials[ORE] >= self.blueprint['b']: 
                self.bots[CLAY] += 1
                self.materials[ORE] -= self.blueprint['b']
                return True
        elif bot == OBSIDIAN:
            if startMaterials[ORE] >= self.blueprint['c'] and startMaterials[CLAY] >= self.blueprint['d']:
                self.bots[OBSIDIAN] += 1
                self.materials[ORE] -= self.blueprint['c']
                self.materials[CLAY] -= self.blueprint['d']
                return True
        elif bot == GEODE:
            if startMaterials[ORE] >= self.blueprint['e'] and startMaterials[OBSIDIAN] >= self.blueprint['f']:
                self.bots[GEODE] += 1
                self.materials[ORE] -= self.blueprint['e']
                self.materials[OBSIDIAN] -= self.blueprint['f']
        return False
    
    def collectMaterials(self, startBots):
        for k,v in startBots.items():
            self.materials[k] += v


     


#tests
tp = Puzzle(testData)
#print(tp.blueprints)
tg = Game(tp.blueprints[0])
print(tg.playGame())



{'id': 1, 'a': 4, 'b': 2, 'c': 3, 'd': 14, 'e': 2, 'f': 7}
Bot value: {'obsidian': 0, 'clay': 0, 'ore': 0}
('obsidian', 0)
T=1 Bots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0}
Bot value: {'obsidian': 0, 'clay': 0, 'ore': 0}
('obsidian', 0)
T=2 Bots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 2, 'clay': 0, 'obsidian': 0, 'geoode': 0}
Bot value: {'obsidian': 0, 'clay': 100, 'ore': 0}
('clay', 100)
T=3 Bots: {'ore': 1, 'clay': 1, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0}
Bot value: {'obsidian': 0, 'clay': 0, 'ore': 0}
('obsidian', 0)
T=4 Bots: {'ore': 1, 'clay': 1, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 2, 'clay': 1, 'obsidian': 0, 'geoode': 0}
Bot value: {'obsidian': 0, 'clay': 0.9285714285714286, 'ore': 0}
('clay', 0.9285714285714286)
T=5 Bots: {'ore': 1, 'clay': 2, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 1, 'clay': 2, 'obsidian': 0,

In [33]:
tg2 = Game(tp.blueprints[1])
print(tg2.playGame())

{'id': 2, 'a': 2, 'b': 3, 'c': 3, 'd': 8, 'e': 3, 'f': 12}
Bot value: {'obsidian': 0, 'clay': 0, 'ore': 0}
('obsidian', 0)
T=1 Bots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0}
Bot value: {'obsidian': 0, 'clay': 0, 'ore': 0}
('obsidian', 0)
T=2 Bots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 2, 'clay': 0, 'obsidian': 0, 'geoode': 0}
Bot value: {'obsidian': 0, 'clay': 0, 'ore': 0}
('obsidian', 0)
T=3 Bots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 3, 'clay': 0, 'obsidian': 0, 'geoode': 0}
Bot value: {'obsidian': 0, 'clay': 100, 'ore': 0}
('clay', 100)
T=4 Bots: {'ore': 1, 'clay': 1, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0}
Bot value: {'obsidian': 0, 'clay': 0, 'ore': 0}
('obsidian', 0)
T=5 Bots: {'ore': 1, 'clay': 1, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 2, 'clay': 1, 'obsidian': 0, 'geoode': 0}
Bot value: {'obs